# Generate pymol script
The scripts can be found in `Data/SARS-CoV-2_spike/Structures/` as `script_fixed.pml` and `script_parallel.pml`

# Important
The command line tool `muscle` is used for multiple sequence alignment. Make sure the executable can be found in the `PATH`.

In [1]:
import os
import shutil
import json
from io import StringIO
from urllib import request
from subprocess import Popen, PIPE

import pandas as pd
from Bio import SeqIO
from Bio.PDB import PDBList

DATA_DIR = "Data"
REFERENCE = "EPI_ISL_402124"

PDB_SERVER = PDBList(verbose=False)
PDB_ID = { 
    "SARS-CoV-2": {
        "spike": "7DF3"
    } 
}
PROTEIN_DOMAIN = {
    "SARS-CoV-2": {
        "spike": {
            "RBD": [s for s in range(319, 541 + 1)],
            "NTD": [s for s in range(13, 303 + 1)],
            "S2": [s for s in range(686, 1273 + 1)],
            "FP": [s for s in range(816, 855 + 1)],
            "HR1": [s for s in range(920, 970 + 1)],
            "HR2": [s for s in range(1163, 1202 + 1)]
        }
    }
}

SITES_COLOR = {
    "fixation": "gold",
    "parallel": "red", 
    "paraFix": "red",
}
DOMAIN_COLOR = {
    "Seq": "0xBCBDBD",
    "NTD": "0x9BCAC8",
    "RBD": "0xC6716B",
    "S2": "0x9EC9A1",
    "FP": "0xBAD3E1",
    "HR1": "0xF0D0CE",
    "HR2": "0xF0D0CE"
    
#     "NTD": "blue",
#     "RBD": "magenta",
#     "S2": "green",
#     "FP": "orange",
#     "HR1": "purple",
#     "HR2": "purple"
}

# SARS-CoV-2 spike protein

Generate pymol script for SARS-CoV-2 spike protein

In [2]:
virus = "SARS-CoV-2"
protein = "spike"

inputDir = os.path.join(DATA_DIR, virus + "_" + protein)

outdirPath = os.path.join(inputDir, "Structures")
if os.path.exists(outdirPath):
    shutil.rmtree(outdirPath)
os.mkdir(outdirPath)

In [3]:
# Download the corresponding PDB file
pdbFilePath = PDB_SERVER.retrieve_pdb_file(
    pdb_code=PDB_ID[virus][protein],
    file_format="pdb",
    pdir=outdirPath
)

# Find the 'DBREF' entry in the PDB file for sites mapping
sitesMapping = []
with open(pdbFilePath, 'r') as f:
    monomer = True
    monomer_ids = set()
    previous_e = -1
    completeRef = False
    for line in f:
        if line.startswith("DBREF"):
            if line.startswith("DBREF1 "):
                completeRef = False
                chain_id, pdb_s, pdb_e = (
                    line[12],
                    int(line[14:18]),
                    int(line[20:24]) + 1,
                )
            elif line.startswith("DBREF2 "):
                completeRef = True
                chain_id, ref_s, ref_e, uniprot_id = (
                    line[12],
                    int(line[45:55]),
                    int(line[57:67]) + 1,
                    line[18:40].strip()
                )
            if line.startswith("DBREF "):
                completeRef = True
                chain_id, pdb_s, pdb_e, ref_s, ref_e, uniprot_id = (
                    line[12],
                    int(line[14:18]),
                    int(line[20:24]) + 1,
                    int(line[55:60]),
                    int(line[62:67]) + 1,
                    line[33:41].strip(),
                )
            if completeRef:
                if previous_e > ref_s:
                    monomer = False
                previous_e = ref_e
                if monomer:
                    monomer_ids.add(chain_id)
                    for pdb_p, ref_p in zip(range(pdb_s, pdb_e), range(ref_s, ref_e)):
                        sitesMapping.append({"uniprotSite": ref_p, "chain": chain_id, "pdbSite": pdb_p})
    sitesMappingAll = pd.DataFrame.from_records(sitesMapping)

In [4]:
# Download the uniprot sequence
res = request.urlopen("https://www.uniprot.org/uniprot/" + uniprot_id + ".fasta")
uniprotSeq = SeqIO.read(StringIO(res.read().decode("utf-8")), "fasta")
uniprotSeq.id = uniprotSeq.description.split('|')[1]
uniprotSeq.description = "Reference"

# Combine the sequences for multiple sequence alignment
sequences = [record for record in SeqIO.parse(os.path.join(inputDir, "selected_AA.fasta"), "fasta")]
refAddedFilePath = os.path.join(outdirPath, "refAdded.fasta")
SeqIO.write(
    sequences=[*sequences, uniprotSeq],
    handle=refAddedFilePath,
    format="fasta"
)

21

In [5]:
# Multiple sequence alignment using muscle
process = Popen(
    ["muscle", "-in", refAddedFilePath, "-diags"],
    stdout=PIPE,
    stderr=PIPE
)
stdout, stderr = process.communicate()
if process.returncode:
    raise BaseException("muscle: " + stderr.decode("UTF-8"))
    
# Map the site numbering between PDB DBREF (uniprot) and the reference
seqs = SeqIO.to_dict(SeqIO.parse(StringIO(stdout.decode("UTF-8")), "fasta"))
refSite, uniprotSite = 0, 0
sitesMapping = []
for n, (ref_aa, uniprot_aa) in enumerate(zip(seqs[REFERENCE].seq, seqs[uniprot_id].seq), start=1):
    validSite = False
    if ref_aa != "-":
        refSite += 1
        validSite = True
    if uniprot_aa != "-":
        uniprotSite += 1
        validSite = True
    if validSite:
        sitesMapping.append({"refSite": refSite, "uniprotSite": uniprotSite, "alignedSite": n})
sitesMapping = pd.DataFrame.from_records(sitesMapping)

In [6]:
sitesMappingAll = pd.merge(sitesMappingAll, sitesMapping, on="uniprotSite", how="left")
# sitesMappingAll = sitesMappingAll.merge( 
#     genomeSitesMapping[genomeSitesMapping["product"] == protein],
#     left_on="refSite",
#     right_on="aaPos",
#     how="left"
# )

In [7]:
sitesMappingAll

,uniprotSite,chain,pdbSite,refSite,alignedSite
0,1,A,1,1,1
1,2,A,2,2,2
2,3,A,3,3,3
3,4,A,4,4,4
4,5,A,5,5,5
...,...,...,...,...,...
1203,1204,A,1204,1204,1204
1204,1205,A,1205,1205,1205
1205,1206,A,1206,1206,1206
1206,1207,A,1207,1207,1207


In [8]:
# The sites (numbering based on EPI_ISL_402125) to be marked on the structure
targetSites = {}
chains = set()
with open(os.path.join(inputDir, "targetSites.json")) as f:
    targetSites = json.load(f)
    
for siteType, sites in targetSites.items():
    sites = sitesMappingAll[sitesMappingAll["refSite"].isin(sites)]
    sites = sites[["chain", "pdbSite"]].drop_duplicates()
    targetSites[siteType] = sites
    chains = chains.union(sites["chain"].unique())

In [9]:
pmlSetView = "set_view (\
     0.787351847,   -0.214461818,    0.577996790,\
     0.602337480,    0.067778848,   -0.795357168,\
     0.131397858,    0.974372327,    0.182546288,\
     0.000000000,    0.000000000, -539.911254883,\
   163.201431274,  163.200393677,  166.706909180,\
  -16535.880859375, 17615.703125000,  -20.000000000 )"

In [10]:
pmlScript = [
    "delete *",
    "fetch {}".format(PDB_ID[virus][protein]),
    "hide",
    "bg_color white",
    "show surface",
    "set transparency, 0.9",
    "set surface_color, grey",
    "; ".join(["show cartoon, c. " + c for c in chains]),
    "color grey"
]

for domain, sites in PROTEIN_DOMAIN[virus][protein].items():
    sites = sitesMappingAll.loc[sitesMappingAll["refSite"].isin(sites), ["chain", "pdbSite"]]
    sel = " or ".join([f"c. {c} and i. {i}" for _, c, i in sites.itertuples()])
    col = DOMAIN_COLOR[domain]
    pmlScript.extend([
        f"select {domain}, {sel}",
        f"color {col}, {domain}",
        f"set surface_color, {col}, {domain}",
        f"set transparency, 0.75, {domain}"
#         f"create Iso_{domain}, {domain}",
#         f"show surface, Iso_{domain}",
#         f"set surface_color, {col}, Iso_{domain}",
#         f"set transparency, 0.1, Iso_{domain}"
    ])

# Script for fixation sites
for category, sites in targetSites.items():
    if category == "fixation":
        sel = " or ".join([f"c. {c} and i. {i}" for _, c, i in sites.itertuples()])
        col = SITES_COLOR[category]
        pmlScript.extend([
            f"select {category}, {sel}",
            f"color {col}, {category}",
            f"show sphere, {category}",
            f"disable {category}"
        ])
        pmlScript.append("reset")
        pmlScript.append(pmlSetView)
        # For the site's label
        label = " or ".join([f"c. {c} and i. {i} and n. CA" for _, c, i in sites.itertuples()])
        pmlScript.extend([
            "set label_position,(5,7,100)",
#             "set label_size, 10",
            "set label_connector, true",
            "set label_bg_color, white",
            "set label_bg_transparency, 0.2",
            f"label {label}, resi\n"
        ])

with open(os.path.join(outdirPath, "script_fixed.pml"), 'w') as f:
    f.write("; ".join(pmlScript))

In [11]:
pmlScript = [
    "delete *",
    "fetch {}".format(PDB_ID[virus][protein]),
    "hide",
    "bg_color white",
    "show surface",
    "set transparency, 0.9",
    "set surface_color, grey",
    "; ".join(["show cartoon, c. " + c for c in chains]),
    "color grey"
]

for domain, sites in PROTEIN_DOMAIN[virus][protein].items():
    sites = sitesMappingAll.loc[sitesMappingAll["refSite"].isin(sites), ["chain", "pdbSite"]]
    sel = " or ".join([f"c. {c} and i. {i}" for _, c, i in sites.itertuples()])
    col = DOMAIN_COLOR[domain]
    pmlScript.extend([
        f"select {domain}, {sel}",
        f"color {col}, {domain}",
        f"set surface_color, {col}, {domain}",
        f"set transparency, 0.75, {domain}"
#         f"create Iso_{domain}, {domain}",
#         f"show surface, Iso_{domain}",
#         f"set surface_color, {col}, Iso_{domain}",
#         f"set transparency, 0.1, Iso_{domain}"
    ])
    
# Script for parallel sites
for category, sites in targetSites.items():
    if category == "parallel":
        sel = " or ".join([f"c. {c} and i. {i}" for _, c, i in sites.itertuples()])
        col = SITES_COLOR[category]
        pmlScript.extend([
            f"select {category}, {sel}",
            f"color {col}, {category}",
            f"show sphere, {category}",
            f"disable {category}"
        ])
        pmlScript.append("reset")
        pmlScript.append(pmlSetView)
        # For the site's label
        label = " or ".join([f"c. {c} and i. {i} and n. CA" for _, c, i in sites.itertuples()])
        pmlScript.extend([
            "set label_position,(5,7,100)",
#             "set label_size, 10",
            "set label_connector, true",
            "set label_bg_color, white",
            "set label_bg_transparency, 0.2",
            f"label {label}, resi\n"
        ])

with open(os.path.join(outdirPath, "script_parallel.pml"), 'w') as f:
    f.write("; ".join(pmlScript))